# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-07-06 23:56:45] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=37606, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=113661651, constrained_json_whitespace_p

[2025-07-06 23:56:57] Attention backend not set. Use fa3 backend by default.
[2025-07-06 23:56:57] Init torch distributed begin.


[2025-07-06 23:56:58] Init torch distributed ends. mem usage=0.00 GB


[2025-07-06 23:57:00] Load weight begin. avail mem=77.48 GB


[2025-07-06 23:57:00] Using model weights format ['*.safetensors']
[2025-07-06 23:57:00] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-07-06 23:57:01] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=60.28 GB


[2025-07-06 23:57:02] INFO:     Started server process [4003357]
[2025-07-06 23:57:02] INFO:     Waiting for application startup.
[2025-07-06 23:57:02] INFO:     Application startup complete.
[2025-07-06 23:57:02] INFO:     Uvicorn running on http://0.0.0.0:37606 (Press CTRL+C to quit)


[2025-07-06 23:57:02] INFO:     127.0.0.1:56400 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-06 23:57:03] INFO:     127.0.0.1:56402 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-06 23:57:03] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-07-06 23:57:04] INFO:     127.0.0.1:56408 - "POST /generate HTTP/1.1" 200 OK
[2025-07-06 23:57:04] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-06 23:57:07] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-07-06 23:57:08] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.16, #queue-req: 0


[2025-07-06 23:57:08] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 121.55, #queue-req: 0


[2025-07-06 23:57:08] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 120.77, #queue-req: 0
[2025-07-06 23:57:08] INFO:     127.0.0.1:58306 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {
    "model_path",
    "is_generation",
    "tokenizer_path",
    "preferred_sampling_params",
}

[2025-07-06 23:57:08] INFO:     127.0.0.1:58318 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-07-06 23:57:08] INFO:     127.0.0.1:58322 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-07-06 23:57:08] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-07-06 23:57:09] INFO:     127.0.0.1:58336 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-07-06 23:57:09] INFO:     127.0.0.1:58348 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-07-06 23:57:09] Cache flushed successfully!
[2025-07-06 23:57:09] INFO:     127.0.0.1:58362 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-07-06 23:57:09] Start update_weights. Load format=auto
[2025-07-06 23:57:09] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-07-06 23:57:09] Using model weights format ['*.safetensors']


[2025-07-06 23:57:10] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.68it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.67it/s]

[2025-07-06 23:57:10] Update weights end.
[2025-07-06 23:57:10] Cache flushed successfully!
[2025-07-06 23:57:10] INFO:     127.0.0.1:58376 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-07-06 23:57:10] Start update_weights. Load format=auto
[2025-07-06 23:57:10] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-07-06 23:57:10] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-07-06 23:57:10] INFO:     127.0.0.1:58380 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-07-06 23:57:10] Child process unexpectedly failed with exitcode=9. pid=4004214
[2025-07-06 23:57:10] Child process unexpectedly failed with exitcode=9. pid=4003803


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-06 23:57:20] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=36396, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=459239456, con

[2025-07-06 23:57:32] Downcasting torch.float32 to torch.float16.


[2025-07-06 23:57:32] Overlap scheduler is disabled for embedding models.
[2025-07-06 23:57:32] Downcasting torch.float32 to torch.float16.
[2025-07-06 23:57:32] Attention backend not set. Use fa3 backend by default.
[2025-07-06 23:57:32] Init torch distributed begin.


[2025-07-06 23:57:33] Init torch distributed ends. mem usage=0.00 GB


[2025-07-06 23:57:34] Load weight begin. avail mem=62.07 GB


...

Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.18s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.28s/it]

[2025-07-06 23:57:38] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=58.05 GB, mem usage=4.02 GB.
[2025-07-06 23:57:38] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-07-06 23:57:38] Memory pool end. avail mem=57.23 GB


[2025-07-06 23:57:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.13 GB


[2025-07-06 23:57:39] INFO:     Started server process [4006429]
[2025-07-06 23:57:39] INFO:     Waiting for application startup.
[2025-07-06 23:57:39] INFO:     Application startup complete.
[2025-07-06 23:57:39] INFO:     Uvicorn running on http://0.0.0.0:36396 (Press CTRL+C to quit)
[2025-07-06 23:57:39] INFO:     127.0.0.1:36742 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-06 23:57:40] INFO:     127.0.0.1:36750 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-06 23:57:40] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-07-06 23:57:40] INFO:     127.0.0.1:36758 - "POST /encode HTTP/1.1" 200 OK
[2025-07-06 23:57:40] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-07-06 23:57:44] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-07-06 23:57:44] INFO:     127.0.0.1:55310 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-07-06 23:57:44] Child process unexpectedly failed with exitcode=9. pid=4006926
[2025-07-06 23:57:44] Child process unexpectedly failed with exitcode=9. pid=4006707


## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-06 23:57:55] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='BAAI/bge-reranker-v2-m3', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=30537, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=894408684, constrained_json_whitespace_pattern=None,

[2025-07-06 23:58:05] Downcasting torch.float32 to torch.float16.


[2025-07-06 23:58:06] Overlap scheduler is disabled for embedding models.
[2025-07-06 23:58:06] Downcasting torch.float32 to torch.float16.
[2025-07-06 23:58:06] Init torch distributed begin.


[2025-07-06 23:58:06] Init torch distributed ends. mem usage=0.00 GB


[2025-07-06 23:58:08] Load weight begin. avail mem=61.47 GB


...

[2025-07-06 23:58:10] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=58.24 GB


[2025-07-06 23:58:11] INFO:     Started server process [4009208]
[2025-07-06 23:58:11] INFO:     Waiting for application startup.
[2025-07-06 23:58:11] INFO:     Application startup complete.
[2025-07-06 23:58:11] INFO:     Uvicorn running on http://0.0.0.0:30537 (Press CTRL+C to quit)


[2025-07-06 23:58:11] INFO:     127.0.0.1:40974 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-06 23:58:12] INFO:     127.0.0.1:40986 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-06 23:58:12] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-07-06 23:58:12] INFO:     127.0.0.1:40994 - "POST /encode HTTP/1.1" 200 OK
[2025-07-06 23:58:12] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-07-06 23:58:16] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-07-06 23:58:16] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-07-06 23:58:16] INFO:     127.0.0.1:35774 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

[2025-07-06 23:58:16] Child process unexpectedly failed with exitcode=9. pid=4009765
[2025-07-06 23:58:16] Child process unexpectedly failed with exitcode=9. pid=4009489


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-06 23:58:28] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=30209, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed

[2025-07-06 23:58:39] Overlap scheduler is disabled for embedding models.
[2025-07-06 23:58:40] Attention backend not set. Use flashinfer backend by default.
[2025-07-06 23:58:40] Init torch distributed begin.


[2025-07-06 23:58:41] Init torch distributed ends. mem usage=0.00 GB


[2025-07-06 23:58:42] Load weight begin. avail mem=78.50 GB


[2025-07-06 23:58:42] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-06 23:58:46] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-07-06 23:58:46] Memory pool end. avail mem=61.66 GB


[2025-07-06 23:58:47] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=60.58 GB


[2025-07-06 23:58:48] INFO:     Started server process [4011674]
[2025-07-06 23:58:48] INFO:     Waiting for application startup.
[2025-07-06 23:58:48] INFO:     Application startup complete.
[2025-07-06 23:58:48] INFO:     Uvicorn running on http://0.0.0.0:30209 (Press CTRL+C to quit)


[2025-07-06 23:58:48] INFO:     127.0.0.1:46980 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-06 23:58:49] INFO:     127.0.0.1:46990 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-06 23:58:49] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-07-06 23:59:54] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-07-06 23:59:54] INFO:     127.0.0.1:46998 - "POST /encode HTTP/1.1" 200 OK
[2025-07-06 23:59:54] The server is fired up and ready to roll!


[2025-07-07 00:00:08] INFO:     127.0.0.1:40682 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

[2025-07-07 00:00:08] Child process unexpectedly failed with exitcode=9. pid=4012687


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 00:00:19] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=35256, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=256682647, constrained_json_whitespace_pattern=None,

[2025-07-07 00:00:30] Attention backend not set. Use flashinfer backend by default.
[2025-07-07 00:00:30] Init torch distributed begin.


[2025-07-07 00:00:30] Init torch distributed ends. mem usage=0.00 GB


[2025-07-07 00:00:32] Load weight begin. avail mem=76.30 GB
[2025-07-07 00:00:32] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-07-07 00:00:40] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=47.28 GB


[2025-07-07 00:00:41] INFO:     Started server process [4018883]
[2025-07-07 00:00:41] INFO:     Waiting for application startup.
[2025-07-07 00:00:41] INFO:     Application startup complete.
[2025-07-07 00:00:41] INFO:     Uvicorn running on http://0.0.0.0:35256 (Press CTRL+C to quit)


[2025-07-07 00:00:41] INFO:     127.0.0.1:57438 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-07 00:00:42] INFO:     127.0.0.1:57446 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 00:00:42] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-07-07 00:00:43] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1c-gpu-23/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-07-07 00:00:56] Resetting ExpertDistributionRecorder...
[2025-07-07 00:00:56] INFO:     127.0.0.1:34072 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-07 00:00:57] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 8, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-07-07 00:00:57] INFO:     127.0.0.1:57448 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 00:00:57] The server is fired up and ready to roll!


[2025-07-07 00:00:57] INFO:     127.0.0.1:57602 - "POST /generate HTTP/1.1" 200 OK


[2025-07-07 00:00:57] INFO:     127.0.0.1:57616 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-07 00:00:59] Write expert distribution to /tmp/expert_distribution_recorder_1751846459.5207353.pt
[2025-07-07 00:00:59] Resetting ExpertDistributionRecorder...
[2025-07-07 00:00:59] INFO:     127.0.0.1:57618 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)

[2025-07-07 00:00:59] Child process unexpectedly failed with exitcode=9. pid=4019677
